# Dimensionality Reduction

When we have many instances, our model trains awfully slow and it's harder to find the answer.This is called Dimensionality Curse.To solve it we can clean up the data set(for MNIST delete the border pixels and merge some), but it will make our model perfom worse and make piplines harder to maintain).Bust it's a wonderful solution if we need to visualize high-dimensional data.The best techniques to reduce deminsions are:<br>
*PCA, Kernel PCA and LLE*

When there is a large distance between instances, we can't build a proper model, because it's difficult ot find correlation.We must reduce Dimensions.There are to main ways to do it: projection(PCA) and Manifold Learning.<br>
But projection doesn't always work.It can be impossible ot safely project data on just single plain.For example, the figure like the Swiss Roll toy can be unfolded istead of projected, and that is just what Manifold learning does.The projection can't be twisted, but the second method works well for twisted data.<br>
But it's not always a good idea to reduce dimensions, because correlations can sometiems be better in higher dimensions.

## PCA(Pricipal Component Analysis)

The main idea of PCA is, as descussed above, a projection.PCA takes the biggest difference between values in one plane as PC1, which answers for highest distribution, and PC2, the second highest distribution across the second line.According to these, we can plot a graph or look for correlation between datapoints.The datapoints, which are alike, are clustured together.

First, we need to select good hyperplane, on which we will project our data.To do that we must look at how well our variance is preserved.We have less loss of the information.To choose this hyperplane we compute mean squared errors to see the minimum.<br>
There is a way to compute our PC-s, it's to use Singular Value Decomposition around the centered data.

There are two ways how to find the best-fit PC-s.First one is old usual distance between the line and the points(Squared Distances), but this time we use the second one, which is distance between the projected point and the origin.It's easier to compute, so we use this(they are the same, just in that case we don't minimaze the distance, but maximaze it, because the farther away the projected point is from origin, the better).The algorithm computes the squared distance(so that negative values don't cancel out the result).<br>
So, we fit the line.Now we measure the slope.This slope tells us along which axis data is mostly distributed.Fore example, for each 4 points in x axis we get 1 point in y-axis(slope is 0.25).So, slope tells us the importance of the value in variance.Using Pythogorian theorem, we can measure the hypotenuse.This hypotenuse is the vector.Butd SVD actually normilizes the length, so that hypotenuse will be one, and other measurments(catets) will be scaled by hypotenuse.The proportions will change, but the ration will remain the same.This scaled vector is called *eigen vector* or *singular vector* and the proportions of the values used to determine this eigenvector are called *loading scores*.A little bit more terminology: the Sum of Squared distances as *eigenvalue* and the root of that eigenvalue a *singular value*.Note:eigenvector just tells us how much of variables is containedin PC, so it's note the same as eigenvalue.<br>
This line is PC-1.We take perpendicular line to that one, which will be PC-2.We can continue to do so for all other PCs.But the merit of doing so can be measured via scree plot, that is the bar plot that shows how much data variation each PC represents.We can compute this variation using eigenvector / number of instances(in other words, the eigenvalue), projecting data points on the line and measuring the distance between origin and this point.The variation will tell us the proportion of the data.In 3-D we look for PC-2 the same way we did in 2-D for PC1 and then take the orthogonal line to that two.<br>
This way we can also draw a neat graph using PCs and projected datapoints(the same datapoints projected on both axis).<br>
Also, I should make it clear what variation is.<br>
[video](https://www.youtube.com/watch?v=FgakZw6K1QQ)

Variation in essence is the sum of distances between the mean and datapoints, just how we used earlier.But there is also a related thing, called covariance.Covariance is the first thing we need to compute correlation.If we plot the data according to all axis and try to fit the line between them we will recieve a trend of the data, which may not exist, or may be positive or negative.It's all covariance tells us.It can tell us which kind of correlation between variables we have.For other ways it's useless, because if we measure covariance on the same datapoints multiplied by two, we will recieve different covariance.So, in total, covariance is sensetive to the scale.And to build correlation we must solve this problem, scaling it.<br>
Covariance can be computed the following way: (sum(x - x_mean) * (y - y_mean) )/ number of instances).For more variables it's the same, we just multiply by other variable difference between the mean and datapoint.The means are GODS for covariance.It selects the correlation type(if slope is positive or negative) thanks to it: if data point are above one mean and below the other, than it's a negative correlation(different sighns), on other hand it's a positive.
[video](https://www.youtube.com/watch?v=qtaqvPAeEJY)

In [3]:
import numpy as np
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [4]:
# The data must be centered, because we must know what exactly mean(the origin) is.
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
# We must transpose the matrix
PC1 = Vt.T[0, 1]
PC2 = Vt.T[0, 2]

In [5]:
# project the data
D2 = Vt.T[:, :2]  ## PCs for all three variables that tells us how to project 3D on 2D
ProjectedX = X_centered.dot(D2)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
Too_projectedX = pca.fit_transform(X)
pca.components_.T[:, 0] # first PC

array([-0.93636116, -0.29854881, -0.18465208])

In [7]:
pca.explained_variance_ratio_  # how variance is distributed across PC

array([0.84248607, 0.14631839])

### Choose numbers of dimension

The best way is to add include all of PCs, that in total give 95% of variation.If we do PCA for data visualization, it's, of course, 2 and 3.

In [8]:
pca = PCA()
pca.fit(X)
cumulativeSum = np.cumsum(pca.explained_variance_ratio_)
best_number_arguments = np.argmax(cumulativeSum >= 95) + 1
best_number_arguments
# Train again with such number of arguments

1

In [9]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)
# Also, we can plot cumsum(the graph with elbow, where teh x axis is number of dimensions and y-axis is variance)

We can compress the MNIST data, but then decompress it back to original size with *reconstruction error* of 1 - cum_variance.<br>
Also, it's wasteful to save the giant data in memory of our computer, so to use it on the fly or to dot some other trick with data, we can feed the data by portions.

In [10]:
X_reconstruct = pca.inverse_transform(X_reduced)

In [11]:
from sklearn.decomposition import IncrementalPCA

amount_batches = 3
inc_pca = IncrementalPCA(n_components = 3)
for X_batch in np.array_split(X, amount_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X)

In [12]:
# stohatic(randomized) PCA, that selects approximation of first d PCs
pca = PCA(n_components=0.95, svd_solver="randomized")

We can use Kernel PCA to do complex non-linear transformation for high-dimensional dataset.So, it's possible to "unroll" some data, that would be impossible otherwise(for examly, swiss roll).

In [13]:
from sklearn.decomposition import KernelPCA
pca_with_rbf = KernelPCA(gamma=0.4, kernel='rbf', n_components=2)

In [14]:
# select kernel and hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf_with_pca = Pipeline([
    ("decomposition", KernelPCA(n_components=2)),
    ("lg_reg", LogisticRegression())
])

param_grid = [{
    "kernel": ['rbf', 'sigmoid'],
    "gamma_value": np.linspace(0.03, 0.05, 10)
}]

best_from_gridSearch = GridSearchCV(clf_with_pca, param_grid, cv=3)
best_from_gridSearch.fit(X, y)

NameError: name 'y' is not defined

### LLE(Local Linear Embedding) - Manifold learning technique

In [15]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_new_reduced = lle.fit_transform(X);

This technique doesn't tely on projection, but on the nearest negihbors(how they are related) and selects the dimensions that preserves these relationships the best way.

Other techniques:<br>
MDS(Multideminsional Scaling) - treis to preserves the distances<br>
t-SNE(Distributed Stocatic Nearest Neighbour) - clusters data based on the classes(dissimilar are aparat and similar are close).Mostly used to visualize the relationship.<br>
LDA(Linear Discriminant Analysis) - classification algorithm ,that in the the proccess of training finds the best axes that distinguishes the data(like PC).It tries to place instances as far as possible.Usually used as preparation for other classification algorithms.

We can measure dimension reduction algorithm performing projection and then reconstruction measuring the error.

It makes sense to combine two dimensionality reduction algorithm.For example, we can use PCA to quickly reduce the most useless one and than apply LLE to find the best dimensions that can preserve the data.